In [1]:
import pandas as pd
import numpy as np
from typing import Dict, Any, List, Union, Optional
import logging

In [2]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [3]:
def drop_nan_columns(df: pd.DataFrame, columns: List[str]) -> pd.DataFrame:
    """
    Drop rows containing NaN values from the specified columns of a DataFrame.

    Args:
        df (pd.DataFrame): The input DataFrame to process.
        columns (List[str]): A list of column names to check for NaN values.

    Returns:
        pd.DataFrame: The updated DataFrame with NaN-containing rows dropped.

    Raises:
        ValueError: If any of the specified columns are not present in the DataFrame.
    """
    # Validate that all specified columns exist in the DataFrame
    missing_columns = set(columns) - set(df.columns)
    if missing_columns:
        raise ValueError(f"Columns not found in DataFrame: {', '.join(missing_columns)}")

    # Identify columns with NaN values
    nan_columns = df[columns].columns[df[columns].isna().any()].tolist()

    # Log dropped rows if any
    if nan_columns:
        nan_count = df[columns].isna().sum()
        logger.info("Dropping rows with NaN values:")
        for col in nan_columns:
            logger.info(f"  {col}: {nan_count[col]} rows")

    # Drop rows with NaN values in specified columns
    original_row_count = len(df)
    df_cleaned = df.dropna(subset=columns)
    dropped_row_count = original_row_count - len(df_cleaned)

    if dropped_row_count > 0:
        logger.info(f"Total rows dropped: {dropped_row_count}")
    else:
        logger.info("No rows were dropped.")

    return df_cleaned

In [4]:
def remove_duplicates(data: pd.DataFrame) -> pd.DataFrame:
    """
    Remove duplicates from a DataFrame.
    
    Args:
        data (pd.DataFrame): Input DataFrame.
        
    Returns:
        pd.DataFrame: DataFrame with duplicates removed.
    """
    num_duplicates = data.duplicated().sum()
    
    if num_duplicates > 0:
        logging.info(f"{num_duplicates} duplicates found.")
        data = data.drop_duplicates(keep="first").reset_index(drop=True)
        logging.info(f"{num_duplicates} duplicates removed.")
    else:
        logging.info("No duplicates found.")
    
    return data

In [5]:

def create_date_column(df: pd.DataFrame, column_names: List[str], new_col: str) -> pd.DataFrame:
    """
    Creates a new date column in the DataFrame by concatenating the values of three specified columns.

    Args:
        df (pd.DataFrame): The DataFrame containing the data.
        column_names (List[str]): A list of three column names to be concatenated [year, month, day].
        new_col (str): The import commandsname of the new date column to be created.

    Returns:
        pd.DataFrame: The DataFrame with the new date column added.

    Raises:
        ValueError: If the input list doesn't contain exactly three column names or if columns are missing.
    """
    if len(column_names) != 3:
        raise ValueError("column_names must contain exactly three elements: [year, month, day]")

    year_col, month_col, day_col = column_names

    # Check if all required columns exist in the DataFrame
    missing_columns = set(column_names) - set(df.columns)
    if missing_columns:
        raise ValueError(f"Missing columns in DataFrame: {', '.join(missing_columns)}")

    # Create copies to avoid SettingWithCopyWarning
    df = df.copy()

    try:
        # Convert year and day columns to integers
        df[year_col] = df[year_col].astype(float).astype(int)
        df[day_col] = df[day_col].astype(float).astype(int)

        # Concatenate the columns to create a date string
        df[new_col] = (df[year_col].astype(str) + '-' + 
                       df[month_col].astype(str) + '-' + 
                       df[day_col].astype(str))

        # Convert to datetime
        df[new_col] = pd.to_datetime(df[new_col], errors='coerce')

        # Log information about the conversion
        valid_dates = df[new_col].notna().sum()
        logger.info(f"Created new date column '{new_col}'. Valid dates: {valid_dates}/{len(df)}")

    except Exception as e:
        logger.error(f"Error creating date column: {str(e)}")
        raise

    return df

In [6]:
def apply_title_case(text):
    """
    Apply title case to a given string.
    
    Args:
        text: The input string to process.
    
    Returns:
        str: The processed string in title case.
    """
    if pd.isna(text):
        return np.nan
    if not isinstance(text, str):
        logger.warning(f"Non-string value encountered: {text}")
        return str(text)
    return text.title()

In [7]:

def process_outcome_column(df: pd.DataFrame) -> pd.DataFrame:
    """
    Process the 'outcome' column of the DataFrame by applying title case.
    
    Args:
        df (pd.DataFrame): The input DataFrame containing the 'outcome' column.
    
    Returns:
        pd.DataFrame: The DataFrame with the processed 'outcome' column.
    """
    if 'outcome' not in df.columns:
        logger.error("'outcome' column not found in the DataFrame")
        return df

    original_null_count = df['outcome'].isnull().sum()
    
    df['outcome'] = df['outcome'].apply(apply_title_case)
    
    new_null_count = df['outcome'].isnull().sum()
    if new_null_count > original_null_count:
        logger.warning(f"Number of null values in 'outcome' increased from {original_null_count} to {new_null_count}")
    
    non_string_count = df['outcome'].apply(lambda x: not isinstance(x, str) if pd.notna(x) else False).sum()
    if non_string_count > 0:
        logger.warning(f"Found {non_string_count} non-string values in 'outcome' after processing")

    return df

In [8]:
def drop_null_values(df: pd.DataFrame, column_name: str = 'outcome') -> pd.DataFrame:
    """
    Drop rows from the DataFrame where the specified column contains null values.

    Args:
        df (pd.DataFrame): The DataFrame from which to drop rows.
        column_name (str): The name of the column to check for null values. Default is 'outcome'.

    Returns:
        pd.DataFrame: The DataFrame with rows containing null values in the specified column dropped.
    """
    initial_row_count: int = df.shape[0]
    cleaned_df: pd.DataFrame = df.dropna(subset=[column_name])
    final_row_count: int = cleaned_df.shape[0]
    dropped_row_count: int = initial_row_count - final_row_count
    
    if dropped_row_count > 0:
        logger.info(f"Total dropped rows with null values in '{column_name}': {dropped_row_count}")
    else:
        logger.info(f"No rows dropped with null values in '{column_name}'")
    return cleaned_df

In [9]:
def categorize_case(case_type: str, criminal_cases: Optional[List[str]]) -> str:
    """
    Categorize a case as 'Criminal' or 'Civil' based on its type.
    
    Args:
        case_type (str): The type of the case.
        criminal_cases (Optional[List[str]]): List of case types considered as criminal.
        
    Returns:
        str: 'Criminal' if the case type is in the criminal cases list or if criminal_cases is None, 'Civil' otherwise.
    """
    if criminal_cases is None:
        return 'Criminal'
    else:
        return 'Criminal' if case_type in criminal_cases else 'Civil'

In [10]:
def categorize_cases(df: pd.DataFrame, criminal_cases: Optional[List[str]] = None) -> pd.DataFrame:
    """
    Categorize all cases in the DataFrame as 'Criminal' or 'Civil'.
    
    Args:
        df (pd.DataFrame): The input DataFrame containing case data.
        criminal_cases (Optional[List[str]]): List of case types considered as criminal.
            If None, all cases are categorized as 'Criminal'.
        
    Returns:
        pd.DataFrame: DataFrame with an added 'nature' column indicating case nature.
    """
    df['nature'] = df['case_type'].apply(lambda x: categorize_case(x, criminal_cases))

    # Check for presence of both case types
    if 'Criminal' not in df['nature'].values:
        logging.warning("No criminal cases found in the DataFrame.")
    if 'Civil' not in df['nature'].values:
        logging.warning("No civil cases found in the DataFrame.")
    
    return df

In [11]:
def apply_dict(value: Any, dictionary: Dict[str, Union[str, List[Any]]]) -> Union[str, None]:
    """
    Find all keys in a dictionary where the given value matches.

    Args:
        value: The value to search for.
        dictionary: The dictionary to search in.

    Returns:
        A list of keys where the value matches, or None if no matches are found.
    """
    matching_keys = []
    for key, dict_value in dictionary.items():
        if isinstance(dict_value, str) and dict_value == value:
            matching_keys.append(key)
        elif isinstance(dict_value, list) and value in dict_value:
            matching_keys.append(key)
    
    if not matching_keys:
        return None
    elif len(matching_keys) == 1:
        return matching_keys[0]
    else:
        return matching_keys

In [12]:
def generate_case_num(df: pd.DataFrame, court_col: str, caseid_type_col: str, caseid_no_col: str, filed_yyyy_col: str, new_col='case_number') -> pd.DataFrame:
    """
    Generates a case number by concatenating court, caseid_type, caseid_no, and filed_yyyy columns.

    Args:
        df (pd.DataFrame): The dataframe containing the necessary columns.
        court_col (str): The name of the column containing court information.
        caseid_type_col (str): The name of the column containing case ID type.
        caseid_no_col (str): The name of the column containing case ID number.
        filed_yyyy_col (str): The name of the column containing the year the case was filed.
        new_col (str): The name of the new column to be created for the case number. Default is 'case_num'.

    Returns:
        pd.DataFrame: DataFrame with the new case number column.
    """
    df[new_col] = df[court_col] + '/' + df[caseid_type_col] + '/' + df[caseid_no_col] + '/' + df[filed_yyyy_col].astype(str)
    return df

def is_concluded(outcome: str, resolved_outcomes: List[str]) -> int:
    """
    Determine if the case is concluded based on the outcome.
    
    Args:
        outcome (str): The outcome of the case.
        resolved_outcomes (List[str]): List of outcomes considered as resolved.
        
    Returns:
        int: 1 if the case outcome is resolved, 0 otherwise.
    """
    return 1 if outcome in resolved_outcomes else 0

In [13]:
def is_case_registered(outcome: str, activity_date: Union[pd.Timestamp, str], filed_date: Union[pd.Timestamp, str]) -> bool:
    """
    Determine if a case is registered based on its outcome and dates.

    Args:
        outcome (str): The outcome of the case.
        activity_date (Union[pd.Timestamp, str]): The date of the activity.
        filed_date (Union[pd.Timestamp, str]): The date the case was filed.

    Returns:
        bool: True if the case is registered, False otherwise.
    """
    try:
        # Normalize the outcome string
        normalized_outcome = outcome.strip().lower()

        # Check if the outcome indicates registration
        is_registered_outcome = 'case registered/filed' in normalized_outcome 

        # Convert dates to pd.Timestamp if they're strings
        if isinstance(activity_date, str):
            activity_date = pd.to_datetime(activity_date, errors='coerce')
        if isinstance(filed_date, str):
            filed_date = pd.to_datetime(filed_date, errors='coerce')

        # Check if dates are equal
        dates_match = pd.notna(activity_date) and pd.notna(filed_date) and activity_date == filed_date

        is_registered = is_registered_outcome and dates_match

        if is_registered:
            logger.debug(f"Case registered: outcome='{outcome}', activity_date={activity_date}, filed_date={filed_date}")
        
        return is_registered

    except Exception as e:
        logger.error(f"Error in is_case_registered: {e}")
        return False

In [14]:
def process_case_status(df: pd.DataFrame, resolved_outcome: List[str]) -> pd.DataFrame:
    """
    Process the DataFrame to add 'concluded' and 'registered' columns.

    Args:
        df (pd.DataFrame): The input DataFrame containing case information.

    Returns:
        pd.DataFrame: The DataFrame with added 'concluded' and 'registered' columns.

    Raises:
        ValueError: If required columns are missing from the DataFrame.
    """
    required_columns = ['outcome', 'activity_date', 'filed_date']
    missing_columns = set(required_columns) - set(df.columns)
    if missing_columns:
        raise ValueError(f"Missing required columns: {', '.join(missing_columns)}")

    #df['concluded'] = df['outcome'].apply(is_case_concluded)
    df['concluded'] = df['outcome'].apply(lambda x: is_concluded(x, resolved_outcome))
    df['registered'] = df.apply(lambda row: is_case_registered(row['outcome'], row['activity_date'], row['filed_date']), axis=1)

    logger.info(f"Processed {len(df)} cases")
    logger.info(f"Concluded cases: {df['concluded'].sum()}")
    logger.info(f"Registered cases: {df['registered'].sum()}")

    return df

In [15]:
def analyze_court_outcomes(df: pd.DataFrame, start_date: str, end_date: str, outcome: str) -> pd.DataFrame:
    """
    Calculate the number of case outcomes per court within a specified period.
    
    Args:
        df (pd.DataFrame): A pandas DataFrame containing the data.
        start_date (str): The starting date of the period (YYYY-MM-DD format).
        end_date (str): The ending date of the period (YYYY-MM-DD format).
        outcome (str): A column representing the outcome of interest.
        
    Returns:
        pd.DataFrame: A DataFrame showing the number of resolved cases per court and case category.
    """
    try:
        period_start = pd.to_datetime(start_date)
        period_end = pd.to_datetime(end_date)
        
        if period_start > period_end:
            raise ValueError("start_date must be earlier than end_date")
        
        required_columns = {'court', 'broad_case_type', 'activity_date', outcome}
        if not required_columns.issubset(df.columns):
            missing_columns = required_columns - set(df.columns)
            raise KeyError(f"Missing required columns: {missing_columns}")
        
        filtered_cases = df[
            (df['activity_date'] >= period_start) &
            (df['activity_date'] <= period_end) &
            (df[outcome] == 1)
        ]
        
        if filtered_cases.empty:
            logging.warning("No cases found for the given date range and outcome.")
   
        outcome_by_type = (
            filtered_cases
            .groupby(['court', 'broad_case_type'])
            .size()
            .reset_index(name='num_cases')
        )

        result = outcome_by_type.pivot_table(
            index='court', 
            columns='broad_case_type', 
            values='num_cases', 
            fill_value=0
        )
        
        logging.info("Successfully calculated case outcomes per court.")
        return result
    
    except Exception as e:
        logging.error(f"An error occurred: {e}")
        raise

In [16]:
def check_time_limit(age, case_category, concluded, time_lines):
    """
    Check if a case falls within the specified time limit for its category and is concluded.
    
    Parameters:
        age (int): The age of the case in days.
        case_category (str): The category of the case.
        concluded (int): The status of the case conclusion (1 for concluded, 0 otherwise).
        time_lines (dict): A dictionary with case categories as keys and time limits as values.
        
    Returns:
        int: 1 if the case is within the time limit and concluded, otherwise 0.
    """
    time_limit = time_lines.get(case_category, 0)
    return 1 if age <= time_limit and concluded == 1 else 0

In [17]:
def determine_judgment_scheduling(df, cutoff_date):
    judgment_date_set_outcomes = ["Judgment Date Given", "Judgment On Notice", "Judgment Date Set"]
    judgment_delivered_outcomes = ["Grant Revoked", "Judgment Delivered", 
                                   "Judgment Delivered- Acquittal", "Judgment Delivered- Case Closed", 
                                   "Judgment Delivered- Convicted"]
    
    # Preprocessing: Filter and sort the DataFrame upfront
    df_filtered = df[df['outcome'].isin(judgment_date_set_outcomes + judgment_delivered_outcomes)]
    df_filtered = df_filtered.sort_values(by=['case_number', 'activity_date'])
    
    # Initialize columns
    df['judgment_status'] = 'Not Scheduled'
    df['set_date'] = pd.NaT
    df['delivery_date'] = pd.NaT
    df['delivery_category'] = ''
    
    # Filter rows with judgment set outcomes and valid schedule dates
    judgment_set_rows = df_filtered[df_filtered['outcome'].isin(judgment_date_set_outcomes) & 
                                    (df_filtered['next_date'] <= cutoff_date)]
    
    # For each case, find the earliest set date
    earliest_schedule = judgment_set_rows.groupby('case_number').first().reset_index()
    
    # Create dictionaries to map case numbers to their schedule dates and statuses
    case_to_set_date = dict(zip(earliest_schedule['case_number'], earliest_schedule['next_date']))
    case_to_status = {case: 'Scheduled' for case in earliest_schedule['case_number']}
    
    # Update the result dataframe with schedule information
    df['set_date'] = df['case_number'].map(case_to_set_date)
    df['judgment_status'] = df['case_number'].map(case_to_status).fillna('Not Scheduled')
    df['delivery_category'] = df['case_number'].map(case_to_status).fillna('')
    
    # Filter rows with judgment delivered outcomes
    judgment_delivered_rows = df_filtered[df_filtered['outcome'].isin(judgment_delivered_outcomes)]
    
    # Find the first delivery date after set date
    for case_number, group in earliest_schedule.groupby('case_number'):
        set_date = group['next_date'].values[0]
        delivery = judgment_delivered_rows[(judgment_delivered_rows['case_number'] == case_number) & 
                                           (judgment_delivered_rows['activity_date'] >= set_date)]
        
        if not delivery.empty:
            delivery_date = delivery.iloc[0]['activity_date']
            df.loc[df['case_number'] == case_number, 'delivery_date'] = delivery_date
            df.loc[df['case_number'] == case_number, 'judgment_status'] = 'Delivered'
            
            if delivery_date <= set_date:
                df.loc[df['case_number'] == case_number, 'delivery_category'] = 'On Time'
            else:
                df.loc[df['case_number'] == case_number, 'delivery_category'] = 'Delayed'
        else:
            earlier_delivery = judgment_delivered_rows[(judgment_delivered_rows['case_number'] == case_number) & 
                                                       (judgment_delivered_rows['activity_date'] < set_date)]
            if earlier_delivery.empty:
                if cutoff_date >= set_date:
                    df.loc[df['case_number'] == case_number, 'judgment_status'] = 'Delayed'
                    df.loc[df['case_number'] == case_number, 'delivery_category'] = 'Delayed'
            else:
                df.loc[df['case_number'] == case_number, 'delivery_date'] = earlier_delivery.iloc[0]['activity_date']
                df.loc[df['case_number'] == case_number, 'judgment_status'] = 'Delivered'
                df.loc[df['case_number'] == case_number, 'delivery_category'] = 'On Time'
    
    return df[df['set_date'].notna()]


In [18]:
def get_on_time_delivery_proportions(scheduled_cases):
    # Get the final status for each case
    final_status = scheduled_cases.groupby(['court', 'case_number']).last().reset_index()
    
    # Group by court and calculate statistics
    court_stats = final_status.groupby('court').agg({
        'case_number': 'count',
        'delivery_category': lambda x: (x == 'On Time').sum()
    }).rename(columns={
        'case_number': 'total_scheduled',
        'delivery_category': 'delivered_on_time'
    })
    
    # Calculate the proportion
    court_stats['proportion_on_time'] = court_stats['delivered_on_time'] / court_stats['total_scheduled']
    
    return court_stats

In [19]:
def get_case_resolution_proportions(df, case_type_timelines):
  # Convert date columns to datetime
  date_columns = ['filed_date', 'activity_date']
  for col in date_columns:
    df[col] = pd.to_datetime(df[col])

  # Filter cases based on case_type_timelines
  df = df[df['broad_case_type'].isin(HC_PMMU_TIME_LINES.keys())]

  # Function to check if a case is resolved within timeline
  def is_resolved_within_timeline(row):
    timeline = case_type_timelines.get(row['broad_case_type'], float('inf'))
    return row['concluded'] == 1 and row['age'] <= timeline

  # Group by court and case type, then calculate statistics
  stats = df.groupby(['court', 'broad_case_type']).apply(
    lambda x: pd.Series({
      'total_cases': len(x),
      'resolved_within_timeline': sum(x.apply(is_resolved_within_timeline, axis=1)),
      'proportion_resolved': sum(x.apply(is_resolved_within_timeline, axis=1)) / len(x) if len(x) > 0 else 0
    })
  ).reset_index()

  return stats

In [20]:
def create_pivot_table(df, case_type_timelines):
  # Calculate statistics using the modified function
  stats = get_case_resolution_proportions(df, case_type_timelines)

  # Pivot the data
  pivot_table = stats.pivot_table(
      index='court',
      columns='broad_case_type',
      values=['total_cases', 'resolved_within_timeline', 'proportion_resolved']
  )

  # Flatten the column hierarchy
  pivot_table.columns = [f"{col[1]}_{col[0]}" for col in pivot_table.columns]

  return pivot_table

In [21]:
def create_case_resolution_pivot(df, case_type_timelines):
    # Prepare the aggfunc dictionary
    aggfunc = {}
    for case_type, timeline in case_type_timelines.items():
        column = f"{case_type}_{timeline}_days"
        aggfunc[f"{column}_total"] = 'count'
        aggfunc[f"{column}_resolved"] = 'sum'
        aggfunc[f"{column}_proportion"] = lambda x: x.sum() / x.count() if x.count() > 0 else 0

    # Create pivot table
    pivot = pd.pivot_table(
        df,
        values=[f"{case_type}_{timeline}_days" for case_type, timeline in case_type_timelines.items()],
        index=['court'],
        aggfunc=aggfunc
    )

    # Rename columns to match the desired format
    new_columns = []
    for case_type, timeline in case_type_timelines.items():
        for stat in ['total', 'resolved', 'proportion']:
            new_columns.append(f"{case_type}_{timeline}_days_{stat}")
    
    pivot.columns = new_columns

    return pivot

In [ ]:
def get_case_resolution_proportions(df, case_type_timelines):
  # Convert date columns to datetime
  date_columns = ['filed_date', 'activity_date']
  for col in date_columns:
    df[col] = pd.to_datetime(df[col])

  # Filter cases based on case_type_timelines
  df = df[df['broad_case_type'].isin(HC_PMMU_TIME_LINES.keys())]

  # Function to check if a case is resolved within timeline
  def is_resolved_within_timeline(row):
    timeline = case_type_timelines.get(row['broad_case_type'], float('inf'))
    return row['concluded'] == 1 and row['age'] <= timeline

  # Group by court and case type, then calculate statistics
  stats = df.groupby(['court', 'broad_case_type']).apply(
    lambda x: pd.Series({
      'total_cases': len(x),
      'resolved_within_timeline': sum(x.apply(is_resolved_within_timeline, axis=1)),
      'proportion_resolved': sum(x.apply(is_resolved_within_timeline, axis=1)) / len(x) if len(x) > 0 else 0
    })
  ).reset_index()

  return stats
def create_pivot_table(df, case_type_timelines):
  # Calculate statistics using the modified function
  stats = get_case_resolution_proportions(df, case_type_timelines)

  # Pivot the data
  pivot_table = stats.pivot_table(
      index='court',
      columns='broad_case_type',
      values=['total_cases', 'resolved_within_timeline', 'proportion_resolved']
  )

  # Flatten the column hierarchy
  pivot_table.columns = [f"{col[1]}_{col[0]}" for col in pivot_table.columns]

  return pivot_table

In [22]:
file_path = '/home/stanoo/dcrt/data/'
raw_df = pd.read_csv(f'{file_path}API/Hc/hc_23-24_data.csv')

/tmp/ipykernel_222326/830036084.py:2: DtypeWarning: Columns (15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_df = pd.read_csv(f'{file_path}API/Hc/hc_23-24_data.csv')


In [23]:
#rename court_name columns to court
# where the sheduled/next date is outside the evaluation period
cutoff_date = pd.Timestamp('2024-06-30') 
df = df.rename(columns={'court_name': 'court'})
raw_df = create_date_column(raw_df.copy(), ['date_dd', 'date_mon', 'date_yyyy'], 'activity_date')
raw_df = create_date_column(raw_df.copy(), ['filed_dd', 'filed_mon', 'filed_yyyy'], 'filed_date')
raw_df = create_date_column(raw_df.copy(), ['next_dd', 'next_mon','next_yyyy'], 'next_date')
df = raw_df.copy()

df = drop_nan_columns(df, ['date_dd', 'date_mon', 'date_yyyy', 'caseid_type', 'caseid_no',
       'filed_dd', 'filed_mon', 'filed_yyyy', 'case_type', 'comingfor'])

df = remove_duplicates(df)
#Apply title case to the outcomes column
df = process_outcome_column(df)
# missing outcomes
df = drop_null_values(df)
# Add broad case category of civil and criminal
df = categorize_cases(df, CRIMINAL_CASES)
# Add case number to the data
df = generate_case_num(df, 'court', 'caseid_type', 'caseid_no', 'filed_yyyy')
# Add filed and resolved outcomes
df = process_case_status(df, RESOLVED_OUTCOMES)

NameError: name 'df' is not defined

In [31]:
# Add sojar case groupings
df['broad_case_type'] = df['case_type'].apply(lambda x: apply_dict(x, BROAD_CASE_TYPES))

In [32]:
# Remove whitespace and rename terminated 
df['outcome'] = df['outcome'].str.strip()
df['outcome'] = df['outcome'].replace('Terminated/ Struck Out/ Dismissed/Case Closed', 'Terminated')

In [34]:
# Apply the function to create a new column with keys
df['productivity'] = df['outcome'].apply(lambda x: apply_dict(x, MERIT_CATEGORY))
# Add the age column 
df['age'] = (df['activity_date'] - df['filed_date']).dt.days
# Add time lines
df['time_lines'] = df.apply(lambda row: check_time_limit(row['age'], row['broad_case_type'], row['concluded'], HC_PMMU_TIME_LINES), axis=1)

In [37]:
# remove leading and trailing spaces on comingfor
df['comingfor'] = df['comingfor'].str.strip()

# create a new column of 1 if reason_adj is not null and comingfor is not in non_adjourned else 0
df['adjourned'] = (df['reason_adj'].notnull() & df['comingfor'].apply(lambda x: x not in NON_ADJOURNABLE)).astype(int)
# an event is adjournable if non_adjourned is not in comingfor
df['adjournable'] = df['comingfor'].apply(lambda x: x not in NON_ADJOURNABLE).astype(int)

# transferred
df['transfer'] = df['outcome'].apply(lambda x: x in TRANSFERED_CASES)

df['productivity_category'] = df.apply(
    lambda row: 'merit' if row['outcome'] in MERIT_OUTCOMES and row['concluded'] == 1 
    else 'non-merit' if row['outcome'] not in MERIT_OUTCOMES and row['concluded'] == 1 
    else None, axis=1
)

In [ ]:
# Create data for output
filed_cases = analyze_court_outcomes(df, '2023-07-01', '2024-06-30', 'registered')
resolved_cases = analyze_court_outcomes(df, '2023-07-01', '2024-06-30', 'concluded')

In [39]:
# count  filed_cases

monthly_filed_cases = df.groupby(['court','date_mon']).agg({'registered':'sum'}).reset_index()
monthly_concluded_cases = df.groupby(['court','date_mon']).agg({'concluded':'sum'}).reset_index()
concluded_cases = df.groupby('court').agg({'concluded':'sum'}).reset_index()


In [40]:
average_time_to_conclude = df.loc[df['concluded'] == 1].pivot_table(index='court', columns='nature', values='age', aggfunc='mean', fill_value=0).round(2)
pmmu_timelines = df[df['time_lines'] == 1].pivot_table(index='court', columns='broad_case_type', values='time_lines', aggfunc='count', fill_value=0)
total_concluded_per_court = df[df['concluded'] == 1].pivot_table(index='court', columns='broad_case_type', values='time_lines', aggfunc='count', fill_value=0)
resolved_within_pmmu_timeline = pmmu_timelines / total_concluded_per_court


In [41]:
court_productivity = df.pivot_table(index='court', columns='productivity', values='concluded', aggfunc='count', fill_value=0)
court_productivity = court_productivity.rename_axis(columns=None)

In [42]:
productivity_pivot_table = pd.pivot_table(
    df,
    values='concluded',  
    index='court',     
    columns='productivity_category', 
    aggfunc='count',   
    fill_value=0        
).rename(columns={'merit': 'Merit', 'non-merit': 'Non_Merit'})


In [43]:
# create a new column of 1 if reason_adj is not null and comingfor is not in non_adjourned else 0
df['adjourned'] = (df['reason_adj'].notnull() & df['comingfor'].apply(lambda x: x not in NON_ADJOURNABLE)).astype(int)
# an event is adjournable if non_adjourned is not in comingfor
df['adjournable'] = df['comingfor'].apply(lambda x: x not in NON_ADJOURNABLE).astype(int)
adjourned_per_court = df.groupby(['court', 'reason_adj'])['adjourned'].sum().reset_index(name='count')
adjourned = df.groupby('court')['adjourned'].sum().reset_index(name='total_adjourned')
adjournable = df.groupby('court')['adjournable'].sum().reset_index(name='total_adjournable')
# deternine the rate of adjournments
adjourn_proportion = pd.merge(adjourned, adjournable, on=['court'])
adjourn_proportion['adjourn_proportion'] = (adjourn_proportion['total_adjourned']/adjourn_proportion['total_adjournable'])*100

In [ ]:
#remove trailing on judge names
judge_names['judge_1'] = judge_names['judge_1'].str.strip()
# create dataframe of judges only
judge_df = df[df['judge_1'].isin(judge_names['judge_1'])]
judge_productivity = judge_df.pivot_table(index='judge_1', columns='productivity', values='concluded', aggfunc='sum', fill_value=0)
judge_productivity = judge_productivity.rename_axis(columns=None)

In [ ]:
# Assuming you have df and case_type_timelines
pivot_table = create_pivot_table(df, case_type_timelines)
judgement_df = determine_judgment_scheduling(df, cutoff_date)
proportions = get_on_time_delivery_proportions(judgement_df)

In [22]:
#CONSTANTS
CRIMINAL_CASES = [
    'Murder Case',
    'Criminal Revision',
    'Criminal Appeal',
    'Murder - Gender Justice Criminal Case',
    'Criminal Court Martial Appeal',
    'Anti-Corruption and Economic Crimes Revision',
    'Criminal Miscellaneous Application',
    'Criminal Applications', 
    'COA Criminal Appeal'
]
BROAD_CASE_TYPES = {
    'Civil Suit': [
        'Civil Suit',
        'Anti-Corruption and Economic Crimes Suit',
        'Family Originating Summons',
        'Family Civil Case',
        'HCC(OS) Family',
        'Commercial Admiralty',
        'Commercial Matters',
    ],
    
    'Adoption': 'Family Adoption',
    'Divorce': 'Family Divorce Cause',

    'Criminal Application': 'Criminal Miscellaneous Application',

    'Miscellaneous Application': [
        'Civil Case Miscellaneous',
        'Judicial Review Miscellaneous',
        'JR  Petition Miscellaneous',
        'Anti-Corruption and Economic Crimes Miscellaneous',
        'Commercial Miscellaneous',
        'Constitution and Human Rights Petitions Miscellaneous',
        'Family Miscellaneous',
        'Commercial Arbitration',
    ],
    'Judicial Review': [
        'Anti-Corruption and Economic Crime Judicial review',
        'Judicial Review ELC',
        'Judicial Review',
    ],
    'Criminal Revision': [
        'Criminal Revision',
        'Anti-Corruption and Economic Crimes Revision',
    ],
    'Criminal Appeal': [
        'Criminal Appeal',
        'Criminal Court Martial Appeal',
        'Anti-Corruption and Economic Crimes Appeal',
    ],
    'Civil Appeal': [
        'Family Appeal',
        'Civil Appeal',
        'Commercial Appeal',
        'Constitution and Human Rights Election Petition Appeal',
        'Constitution and Human Rights Petition Appeal',
        'Constitution and Human Rights Election Petition Appeal',
        'Gender Justice Civil Appeal',
        'Constitution and Human Rights Miscellaneous Election Petition Appeal (MEPA)',
    ],
    'Constitution Petition': [
        'Anti Corruption and Economic Crimes Petition',
        'High Court Criminal Petition',
        'Constitution and Human Rights Petition (Civil)',
        'Constitution and Human Rights Election Petition',
        'High Court Constitution and Human Rights Petitions (Criminal)',
        'Commercial Petition',
    ],
    'Probate Administration': [
        'Family P&A Intestate',
        'Family P&A Ad Litem',
        'Family P&A Ad Colligenda',
        'Family P&A Citation',
        'Family P&A Testate',
        'Family P&A Resealing of Grant',
        'Family P&A De Bonis Non',
        'Resealing of Grant',
        'Citation-Family',
    ],
    'Murder': [
        'Murder Case',
        'Murder - Gender Justice Criminal Case',
    ],
    'Tax Appeal': [
        'Commercial Income Tax Apperiod_startpeal',
        'Commercial Custom Tax Appeal',
    ],
    'Bankruptcy and Insolvency' : [
        'Commercial Insolvency Notice Petition',
        'Commercial Insolvency Petition',
        'Commercial Bankruptcy Notice',
        'Commercial Insolvency Cause',
        'Commercial Insolvency Notice',
        'Commercial Bankruptcy Cause',
        'Commercial Winding Up Cause',
    ]
}


RESOLVED_OUTCOMES = ['Ruling Delivered- Case Closed',
 'Terminated',
 'Matter Settled- Case Closed',
 'Application Dismissed - Case Closed',
 'Judgment Delivered- Case Closed',
 'Matter Withdrawn',
 'Application Allowed - Case Closed',
 'Application Withdrawn - Case Closed',
 'Judgment Delivered- Convicted',
 'Placed In Probation',
 'Dismissed',
 'Judgment Delivered',
 'Judgment Delivered- Acquittal',
 'Ruling Delivered- Accused Discharged',
 'Abated',
 'Consolidated- Case Closed',
 'Grant Confirmed',
 'Limited Grant Issued',
 'Struck Out',
 'Grant Revoked',
 'Consent Recorded - Case Closed',
 'Dismissed For Want Of Prosecution - Case Closed',
 'Out Of Court Settlement Reached',
 'Appeal Dismissed',
 'Retrial',
 'Appeal Rejected',
 'Sentence Commuted',
 'Ruling Delivered- Application Closed',
 'Probation Orders Issued',
 'Order Issued - Case Closed',
 'Revision Declined']


TRANSFERED_CASES = ['File Transfered -case Closed', 
        'File Transferred',]

MERIT_CATEGORY = {
    'Judgment Delivered': [
        'Judgment Delivered- Case Closed',
        'Judgment Delivered',
        'Judgment Delivered- Acquittal',
        'Judgment Delivered- Convicted',
        'Grant Revoked',
        'Retrial'
        ],
    'Ruling Case Closed': [
        'Ruling Delivered- Case Closed', 
        'Ruling Delivered- Accused Discharged',
        ],
    'Final Grant': [
        'Grant Confirmed',
        'Limited Grant Issued',
        ],
    'Case Withdrawn': [
        'Matter Withdrawn',
        'Application Withdrawn - Case Closed',
        ],
   'Out Of Court Settlement': [
        'Consent Recorded - Case Closed',
        'Matter Settled Through Mediat26	Jun	2024	ion',
        'Out Of Court Settlement Reached',
    ],
    'Dismissed':[
        'Dismissed For Want Of Prosecution - Case Closed',
        'Dismissed',
        'Appeal Dismissed',
        'Terminated'
    ],

    'Case Closed': [
        'Struck Out',
        'Application Dismissed - Case Closed',
        'Application Allowed - Case Closed',
        'Matter Settled- Case Closed',
        'Ruling Delivered- Application Closed',
        'Consolidated- Case Closed',
        'Abated',
        'Placed In Probation',
        'Revision Declined',
        'Probation Orders Issued',
        'Appeal Rejected',
        'Interlocutory Judgement Entered',
        'Order issued - Case closed'   
    ],
}

HC_PMMU_TIME_LINES = {
    'murder': 360,
    'revision': 90,
    'misc_application': 90,
    'suit': 360,
    'judicial_review': 180,
    'constitutional_petition': 180,
}

NON_ADJOURNABLE = [
    'Taxation and Issuance of Certificates',
    'Orders',
    'Appointments of  Mediator',
    'Screening of files for Mediation',
    'Post-judgment',
    'Re-activation',
    'Reactivation',
    'Notice of Taxation',
    'Entering Interlocutory Judgments',
    'Approval by DR', 
    'Registration/Filing-Application', 
    'Registration/Filing', 
    'Registration/Filing-Application',
 ]
MERIT_OUTCOMES = [
    'Ruling Delivered- Case Closed', 
    'Judgment Delivered- Case Closed',
    'Judgment Delivered',
    'Judgment Delivered- Acquittal',
    'Judgment Delivered- Convicted',
    'Grant Revoked',
    'Ruling Delivered- Accused Discharged',
    'Retrial'
]